download the document file.

In [10]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json -q

In [ ]:
!pip install sentence_transformers==2.7.0

Q1. multi-qa-distilbert-cos-v1

In [1]:
model_name = 'multi-qa-distilbert-cos-v1'
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
embedding_model.encode(user_question)[0]

0.078222655

Q2. 

In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
#filtering the whole document to get only ml zooomcamp data.

ml_documents = []
for docs in documents:
    if docs['course'] == 'machine-learning-zoomcamp':
        ml_documents.append(docs)

In [6]:
len(ml_documents)

375

In [7]:
ml_documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [8]:
embedding = []
for doc in ml_documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embed = embedding_model.encode(qa_text)
    embedding.append(embed)

In [9]:
import numpy as np

In [10]:
X = np.array(embedding)

In [11]:
X.shape

(375, 768)

Q3.

In [12]:
v = embedding_model.encode(user_question)

In [13]:
scores = X.dot(v)

In [14]:
np.max(scores)

0.6506573

vector search engine

In [18]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_documents, embeddings=X)
vector_search_engine_result = search_engine.search(v, num_results=5)

In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [34]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [29]:
len(vector_search_engine_result)

5

calculate hit rate.

In [36]:
df_ground_truth[df_ground_truth.document == 'ee58a693']

,question,course,document
2193,"If I join the course late, can I still partici...",machine-learning-zoomcamp,ee58a693
2194,Will I be able to obtain a certificate if I jo...,machine-learning-zoomcamp,ee58a693
2195,How many course projects must I submit to be e...,machine-learning-zoomcamp,ee58a693
2196,What is the requirement for peer reviews to ge...,machine-learning-zoomcamp,ee58a693
2197,Can I submit homework if I join the course aft...,machine-learning-zoomcamp,ee58a693


In [41]:
from tqdm import tqdm

relevance_total = []

for doc in tqdm(ground_truth):
    query_embeddding = embedding_model.encode(doc['question'])
    result = search_engine.search(query_embeddding, num_results=5)
    relevance = [doc['document'] == d['id']  for d in result]

    relevance_total.append(relevance)
    

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [02:17<00:00, 13.28it/s]


In [42]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [43]:
hit_rate(relevance_total)

0.9398907103825137